# LU Factorization

In [1]:
# Library:
# jupyterlab           3.3.1 
# numpy                1.22.3 
# sympy                1.10

import numpy as np  # Library for mathematical operation
import sympy as sy  # Library for printing matrix equation

## Metoda Doolittle’a
$$ A = LU $$

$$
{\begin{bmatrix}a_{11}&a_{12}&\cdots &a_{1n}\\a_{21}&a_{22}&\cdots &a_{2n}\\\vdots &\vdots &\ddots &\vdots \\a_{n1}&a_{n2}&\cdots &a_{nn}\end{bmatrix}}={\begin{bmatrix}1&0&\cdots &0\\l_{21}&1&\cdots &0\\\vdots &\vdots &\ddots &0\\l_{n1}&l_{n2}&\cdots &1\end{bmatrix}}\cdot {\begin{bmatrix}u_{11}&u_{12}&\cdots &u_{1n}\\0&u_{22}&\cdots &u_{2n}\\\vdots &\vdots &\ddots &\vdots \\0&0&\cdots &u_{nn}\end{bmatrix}}
$$

$ u_{ij}=a_{ij}-\sum _{k=1}^{i-1}l_{ik}u_{kj}$ for $j\in \{i,\ i+1,\ldots ,\ n\}$

$ l_{ji}={\frac {1}{u_{ii}}}\left(a_{ji}-\sum _{k=1}^{i-1}l_{jk}u_{ki}\right)$ for $j\in \{i+1,\ i+2,\ldots ,\ n\}$

In [2]:
# Class to do factorization
class LUFactorization:
    @staticmethod
    def DollitleMethod( A ):
        rows, cols = A.shape                      # Get matrix shape rows and cols
        L = np.matrix(np.diag( np.ones( rows ) )) # Create matrix L (1 on diagonal)
        U = np.matrix(np.zeros( A.shape ))        # Create zeros matrix U 
        
        L[:, 0] = A[:, 0] / A[0,0]   # First column in L matrix is first column A / A(0,0)
        U[0, :] = A[0, :]            # First row in U matrix is first column of A

        for i in range( 1, rows ):                # For all rows in matrix
            for j in range( i, cols):             # For all columns in i row
                U[i,j] = A[i,j]-L[i,0:i]*U[0:i,j] # U(i,j) = A(i,j)-sum(L(i,k)*U(k,j)), where k[1,i-1]
                
            
            for j in range( i+1, rows ):                   # For all rows in i column
                L[j,i] = (A[j,i]-L[j,0:i]*U[0:i,i])/U[i,i] # L(j,i) = A(j,i)-sum(L(j,k)*U(k,i)), where k[1,i-1]
    
        return (L, U)


## Substitution
---
* Forward

$${\begin{bmatrix}l_{11}&0&\cdots &0\\l_{21}&l_{22}&\cdots &0\\\vdots &\vdots &\ddots &0\\l_{n1}&l_{n2}&\cdots &l_{nn}\end{bmatrix}}
{{\begin{bmatrix}
y_{1}\\
y_{2}\\
\vdots\\
y_{n}\end{bmatrix}}}=
{{\begin{bmatrix}
b_{1}\\
b_{2}\\
\vdots\\
b_{n}\end{bmatrix}}}$$

$ y_{1} = \frac{b_1}{l_{11}}$

$ y_{i}=\frac{ b_i-\sum _{j=1}^{i-1}l_{ij}y_{j} } {l_{ii}}$ for $i\in \{2,\ldots ,\ n\}$

---
* Backward

$${\begin{bmatrix}u_{11}&u_{12}&\cdots &u_{1n}\\0&u_{22}&\cdots &u_{2n}\\\vdots &\vdots &\ddots &\vdots \\0&0&\cdots &u_{nn}\end{bmatrix}}{{\begin{bmatrix}
y_{1}\\
y_{2}\\
\vdots\\
y_{n}\end{bmatrix}}}=
{{\begin{bmatrix}
b_{1}\\
b_{2}\\
\vdots\\
b_{n}\end{bmatrix}}}$$

$y_n = \frac{b_n}{u_{nn}}$

$y_i = \frac{ b_i-\sum _{j=i+1}^{n}u_{ij}y_{j} } {u_{ii}}$ for $i\in \{1,\ldots ,\ n-1\}$


In [3]:
# Class to do substitution
class Substitution:
    @staticmethod
    def Forward( L, b ):
        rows, cols = b.shape
        y = np.matrix(np.zeros(b.shape))
        y[0,0] = b[0,0]/L[0,0]
        for i in range(1, rows):
            y[i,0] = (b[i,0]-L[i,0:i]*y[0:i,0])/L[i,i]
        return y
    
    @staticmethod
    def Backward( U, b ):
        rows, cols = U.shape
        y = np.matrix(np.zeros(b.shape), dtype=float)
        y[rows-1, 0] = b[rows-1, 0]/U[rows-1, cols-1]
        for i in range(rows-2, -1, -1):
            y[i,0] = (b[i,0]-U[i,i+1:cols]*y[i+1:rows,0])/U[i,i]
        return y
        

In [22]:
# Matrix to LU Factorization

A = np.matrix( [ [1, 2, 3, 4], [-1, 1, 2, 1], [0, 2, 1, 3], [0, 0, 1, 1] ], dtype=float) 
#A = np.matrix( [ [5,3,2],[1,2,0],[3,0,4] ], dtype=float )

In [23]:
# Display result
L, U = LUFactorization.DollitleMethod(A)
A_sy = sy.Matrix(A)
L_sy = sy.Matrix(L.round(3))
U_sy = sy.Matrix(U.round(3))

sy.Eq( A_sy, sy.MatMul(L_sy, U_sy, evaluate=False), evaluate=False )

Eq(Matrix([
[ 1.0, 2.0, 3.0, 4.0],
[-1.0, 1.0, 2.0, 1.0],
[   0, 2.0, 1.0, 3.0],
[   0,   0, 1.0, 1.0]]), Matrix([
[ 1.0,     0,      0,   0],
[-1.0,   1.0,      0,   0],
[   0, 0.667,    1.0,   0],
[   0,     0, -0.429, 1.0]])*Matrix([
[1.0, 2.0,    3.0,    4.0],
[  0, 3.0,    5.0,    5.0],
[  0,   0, -2.333, -0.333],
[  0,   0,      0,  0.857]]))

In [11]:
# Reverse LU=A'
LU = L*U
LU_sy = sy.Matrix(LU.round(3))
sy.Eq( sy.MatMul(L_sy, U_sy, evaluate=False), LU_sy,  evaluate=False )

Eq(Matrix([
[ 1.0,     0,      0,   0],
[-1.0,   1.0,      0,   0],
[   0, 0.667,    1.0,   0],
[   0,     0, -0.429, 1.0]])*Matrix([
[1.0, 2.0,    3.0,    4.0],
[  0, 3.0,    5.0,    5.0],
[  0,   0, -2.333, -0.333],
[  0,   0,      0,  0.857]]), Matrix([
[ 1.0, 2.0, 3.0, 4.0],
[-1.0, 1.0, 2.0, 1.0],
[   0, 2.0, 1.0, 3.0],
[   0,   0, 1.0, 1.0]]))

## Determinant of array
---
$det(A) = det(L*U) = det(L)*det(U)$

For triangular matrix

$ {det(L)} =l_{11}\cdot l_{22}\cdot \ldots \cdot l_{nn} $

$ {det(U)} =u_{11}\cdot u_{22}\cdot \ldots \cdot u_{nn} $

$det(A) = det(U)$

In [12]:
rows, cols = A.shape
detU = U[0,0]
for i in range(1, rows):
    detU = detU*U[i,i]
    
detA_np = np.linalg.det(A)
    
print("Determinant of array det(U)=det(A)=", detU)
print("Determinant of array (numpy) det(A)=", detA_np)

Determinant of array det(U)=det(A)= -6.0
Determinant of array (numpy) det(A)= -6.0


## Solving linear equations

$\mathbf {L} \cdot \mathbf {U} \cdot \mathbf {x} =\mathbf {b}$

so,

$\mathbf {L} \cdot \mathbf {z} =\mathbf {b}$

$\mathbf {U} \cdot \mathbf {x} =\mathbf {z}$

In [13]:
s = np.array( [ ['x1'], ['x2'], ['x3'], ['x4'] ] )
b = np.matrix( [[1],[1],[1],[1]] )

z = Substitution.Forward(L, b)
x = Substitution.Backward(U, z)

x_sy = sy.Matrix( x.round(3) )
s_sy = sy.Matrix( s )

print("Result using Factorization")
sy.Eq( s_sy, x_sy, evaluate=False ) 

Result using Factorization


Eq(Matrix([
[x1],
[x2],
[x3],
[x4]]), Matrix([
[-1.0],
[-1.0],
[   0],
[ 1.0]]))

In [14]:
x_n   = np.linalg.solve(A,b)               
x_n_s = sy.Matrix( x_n.round(3) )
print("Result using numpy")
sy.Eq( s_sy, x_n_s, evaluate=False ) 

Result using numpy


Eq(Matrix([
[x1],
[x2],
[x3],
[x4]]), Matrix([
[-1.0],
[-1.0],
[   0],
[ 1.0]]))

In [25]:
Q, R = np.linalg.qr(A)
print(Q)
print(R)

[[-0.70710678 -0.51449576  0.417957   -0.24618298]
 [ 0.70710678 -0.51449576  0.417957   -0.24618298]
 [-0.         -0.68599434 -0.62693551  0.36927447]
 [-0.         -0.          0.50751922  0.86164044]]
[[-1.41421356 -0.70710678 -0.70710678 -2.12132034]
 [ 0.         -2.91547595 -3.25847312 -4.6304618 ]
 [ 0.          0.          1.97036873  0.71649772]
 [ 0.          0.          0.          0.73854895]]
